In [1]:
import torch
device = torch.device('cuda')

In [2]:
import torch
from torch2trt import TRTModule

model_trt_collision = TRTModule()
model_trt_collision.load_state_dict(torch.load('best_model_trt.pth')) 

<All keys matched successfully>

In [3]:
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

normalize = torchvision.transforms.Normalize(mean, std)

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [ ]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
speed_slider = widgets.FloatSlider(description='speed', min=0.0, max=0.5, value=0.0, step=0.01, orientation='horizontal')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.VBox([widgets.HBox([image]), speed_slider]))

In [ ]:
from jetbot import Robot

robot = Robot()

In [ ]:
categories = ['forward', 'left', 'right', 'stop']
class SimpleDataset:
    def __init__(self, categories):
        self.categories = categories

dataset = SimpleDataset(categories)


In [ ]:
import torch.nn.functional as F
import time

car_command_status = 0
car_command = ''
car_last_command = ''
prediction_position_widget = widgets.Text(description='Prediction')
speed_slider.value = 0.20
score_widgets = []

for category in dataset.categories:
    score_widget = widgets.FloatSlider(min=0.0, max=1.0, description=category, orientation='vertical')
    score_widgets.append(score_widget)
    
def update(change):
    global blocked_slider, robot,car_command_status, car_command, car_last_command
    x = change['new'] 
    x = preprocess(x)
    y = model_trt_collision(x)
    y = F.softmax(y, dim=1).detach().cpu().numpy().flatten()

    category_index = y.argmax()
    prediction_position_widget.value = dataset.categories[category_index]
   

    for i, score in enumerate(list(y)):
        score_widgets[i].value = score
    if car_command_status == 0:
        car_command = dataset.categories[category_index]
        car_command_status = 1

        if car_command_status == 1:
            if car_command == 'forward':
                robot.forward(speed_slider.value)
            elif car_command == 'left':
                robot.left(speed_slider.value)
            elif car_command == 'right':
                robot.right(speed_slider.value)
            else:
                robot.stop()
        car_command_status = 0
        car_command = ''
        
update({'new': camera.value})  # we call the function once to initialize

In [ ]:
display(
    widgets.VBox(
        [
            widgets.HBox(score_widgets),
            image, 
            speed_slider
        ]
    )
)


In [ ]:
camera.observe(update, names='value') 

In [11]:
import time

# camera.unobserve(execute, names='value')

time.sleep(0.1)  

robot.stop()

In [ ]:
camera.stop()